### YOLO v8

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

import pandas as pd
import cv2
import warnings
warnings.filterwarnings("ignore")

from ultralytics import YOLO
from glob import glob
from tqdm import tqdm
from IPython.display import clear_output

In [ ]:
SEED = 42
BATCH_SIZE = 4
MODEL = "v8x"

data_path = "../../../data/"

In [ ]:
model = YOLO("yolov8x.pt")

In [ ]:
results = model.train(
    data= data_path + "yolo/all.yaml",
    imgsz=1024,
    epochs=200,
    batch=BATCH_SIZE,
    patience=10,
    workers=16,
    device=0,
    exist_ok=True,    
    project=f"{MODEL}",
    name=f"train",
    seed=SEED,
    pretrained=False,
    resume=False, 
    optimizer="AdamW",
    lr0=1e-3,
    val=False, 
    cache=True,
    # augmentation
    fliplr=0.0,
    # augmentation 원복
    # translate=0.0,
    # scale=0.0,
    # mosaic=0.0, 
    # save_period=10,
)

In [ ]:
def get_test_image_paths(test_image_paths):    
    for i in range(0, len(test_image_paths), BATCH_SIZE):
        yield test_image_paths[i:i+BATCH_SIZE]

In [ ]:
model = YOLO("v2/train/weights/last.pt")
test_image_paths = glob(data_path + "test/*.png")
for i, image in tqdm(enumerate(get_test_image_paths(test_image_paths)), total=int(len(test_image_paths)/BATCH_SIZE)):
    model.predict(image, imgsz=(1024, 1024), iou=0.2, conf=0.5, save_conf=True, save=False, save_txt=True, project=f"{MODEL}", name="predict",
                  exist_ok=True, device=0, augment=True, verbose=False)
    if i % 5 == 0:
        clear_output(wait=True)

In [ ]:
def yolo_to_labelme(line, image_width, image_height, txt_file_name):    
    file_name = txt_file_name.split("/")[-1].replace(".txt", ".png")
    class_id, x, y, width, height, confidence = [float(temp) for temp in line.split()]
    
    x_min = int((x - width / 2) * image_width)
    x_max = int((x + width / 2) * image_width)
    y_min = int((y - height / 2) * image_height)
    y_max = int((y + height / 2) * image_height)
    
    return file_name, int(class_id), confidence, x_min, y_min, x_max, y_min, x_max, y_max, x_min, y_max

In [ ]:
infer_txts = glob(f"{MODEL}/predict/labels/*.txt")

results = []
for infer_txt in tqdm(infer_txts):
    base_file_name = infer_txt.split("/")[-1].split(".")[0]
    imgage_height, imgage_width = cv2.imread(f"{data_path}test/{base_file_name}.png").shape[:2]        
    with open(infer_txt, "r") as reader:        
        lines = reader.readlines()        
        for line in lines:
            results.append(yolo_to_labelme(line, imgage_width, imgage_height, infer_txt))

df_submission = pd.DataFrame(data=results, columns=["file_name", "class_id", "confidence", "point1_x", "point1_y", "point2_x", "point2_y", "point3_x", "point3_y", "point4_x", "point4_y"])
df_submission.sort_values(["confidence"], ascending=False, inplace=True)
df_submission.to_csv(f"../../../submissions/yolov8.csv", index=False)